In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show
//raw_sentiment.show


+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
////////////////////////////////////////////////////////////
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")//don't delete


val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val randomForest = new RandomForestClassifier()
  .setLabelCol("label")
  .setFeaturesCol(hashingTF.getOutputCol)
  .setNumTrees(10)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, randomForest))



tokenizer = tok_fac2092d69bc
hashingTF = hashingTF_6f91a5d1e0b6
randomForest = rfc_b698db267fe2
pipeline = pipeline_ebfe683b7a32


pipeline_ebfe683b7a32

In [4]:
val model = pipeline.fit(raw_sentiment)


model = pipeline_ebfe683b7a32


pipeline_ebfe683b7a32

In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model") ///

sameModel = pipeline_ebfe683b7a32


pipeline_ebfe683b7a32

In [7]:
val predictionsDF = sameModel.transform(raw_sentiment) ///

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[4.40143807491873...|[0.44014380749187...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[4.91878457686985...|[0.49187845768698...|       1.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[5.39843146807782...|[0.53984314680778...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[5.63794180065782...|[0.56379418006578...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[4.84181507313086...|[0.48418150731308...|       1.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [8]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [9]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [10]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
|  0.559856192508126|
| 0.5081215423130143|
|0.46015685319221716|
|  0.436205819934217|
| 0.5158184926869137|
|  0.512082889033219|
| 0.5155434296793093|
| 0.5497978998840494|
| 0.4921748243566536|
| 0.5062781790298415|
|  0.512082889033219|
|  0.436205819934217|
|0.46015685319221716|
| 0.5772852861978782|
|0.48520840333583826|
| 0.4893550278186282|
| 0.5062781790298415|
|  0.512082889033219|
| 0.5337092238932458|
| 0.5437651845545968|
+-------------------+
only showing top 20 rows

